In [1]:
import pandas as pd
from tqdm import tqdm
from dateutil.parser import parse
import datetime as dt
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('D:/ML/Kis_Api_test/')
from DB_System.communication import StockConnection

In [4]:
sc = StockConnection()

In [5]:
# db 추출
def db_extract(result):
    #name = result['output1']['hts_kor_isnm']
    df1 = pd.DataFrame(result['output2'])
    df1.drop(df1.iloc[:,6:],axis=1,inplace=True)
    df1['hts_kor_isnm'] = result['output1']['hts_kor_isnm']
    df1['stck_shrn_iscd'] = result['output1']['stck_shrn_iscd']
    #df['name'] = name
    return df1

In [6]:
kospi = pd.read_csv('../../../data_0402_20230323.csv', encoding='cp949')
kosdaq = pd.read_csv('../../../data_0457_20230323.csv', encoding='cp949')
kospi = kospi[['단축코드', '한글 종목약명', '상장일']]
kosdaq = kosdaq[['단축코드', '한글 종목약명', '상장일']]
kospi_kosdaq = pd.concat([kospi, kosdaq])

In [7]:
kospi_kosdaq[kospi_kosdaq['단축코드'] == '247540']

,단축코드,한글 종목약명,상장일
835,247540,에코프로비엠,2019/03/05


In [19]:
kospi_kosdaq.to_csv('../../SRC/kospi_kosdaq_information.csv')

In [8]:
dt.datetime.today()

datetime.datetime(2023, 3, 23, 20, 34, 9, 359504)

In [9]:
list(kospi_kosdaq['단축코드']).pop()

'238490'

In [10]:
fin_res = []
companys = list(kospi_kosdaq['단축코드'])
with tqdm(total = len(companys)) as pbar:
        while len(companys) > 0:
            try:
                company = companys.pop()
                
                first_date = parse(kospi_kosdaq[kospi_kosdaq['단축코드'] == company]['상장일'].iloc[0])
                first_date = first_date if first_date > parse('20180101') else parse('20180101')
                last_date = dt.datetime.today()
                term = dt.timedelta(days=98)
                term_bias_date = first_date
                buf2 = []
                while last_date - term_bias_date > term:  
                    date1 = str(term_bias_date)[:10].replace('-', '')
                    date2 = str(term_bias_date + term - dt.timedelta(days=1))[:10].replace('-', '')
        #             time.sleep(0.1)
                    buf = sc.date_price(company, date1, date2)
                    term_bias_date = term_bias_date + term
                    buf2.append(db_extract(buf))   
                buf = sc.date_price(company, str(term_bias_date + dt.timedelta(days=1))[:10].replace('-', ''), str(dt.datetime.today())[:10].replace('-', ''))
                buf2.append(db_extract(buf))
                res = pd.concat(buf2)
                pbar.update(1)
                res.columns =['날짜','종가','시가','최고가','최저가','누적거래량', '종목명', '종목코드']
                res['날짜'] = pd.to_datetime(res['날짜'])
                res.sort_values(by='날짜',inplace=True)
                fin_res.append(res)
                data = pd.concat(fin_res)
            except:
                companys.append(company)
                buf2.clear()

# for company in tqdm(kospi_kosdaq['단축코드']):
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2575/2575 [1:30:16<00:00,  2.10s/it]


In [11]:
len(fin_res)

2575

In [17]:
data.to_csv('../../../df_230323_fin.csv')

In [22]:
first_date = parse(sorted(kospi_kosdaq['상장일'].unique())[0])

### 활용예시 (삼성전자 2022년도 1년치 주가를 저장)

In [9]:
def yyy(company, year):
    result_1Q = sc.date_price(company,f'{year}0101',f'{year}0331','D')
    result_2Q = sc.date_price(company,f'{year}0401',f'{year}0630','D')
    result_3Q = sc.date_price(company,f'{year}0701',f'{year}0930','D')
    result_4Q = sc.date_price(company,f'{year}1001',f'{year}1231','D')
    df_1q = db_extract(result_1Q)
    df_2q = db_extract(result_2Q)
    df_3q = db_extract(result_3Q)
    df_4q = db_extract(result_4Q)
    df = pd.concat([df_1q,df_2q,df_3q,df_4q])
#     print(df)
    df.columns =['날짜','종가','시가','최고가','최저가','누적거래량', '종목명', '종목코드']
    df['날짜'] = pd.to_datetime(df['날짜'])
    df.sort_values(by='날짜',inplace=True)
    return df 

In [10]:
companys = buf3
fin = []
for company in tqdm(companys):
    try:
        buf = []
        for year in [2018, 2019, 2020, 2021, 2022, 2023]:
#             print(f'{company} : {year}')
            
            buf2 = yyy(company, year)
            buf.append(buf2)
        fin.append(pd.concat(buf))
    except:
        pass
res = pd.concat(fin)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2438/2438 [1:16:49<00:00,  1.89s/it]


In [11]:
res2 = res.dropna(axis=0)

In [12]:
res2

,날짜,종가,시가,최고가,최저가,누적거래량,종목명,종목코드
60,2018-01-02,9870,9750,9900,9700,120676,동화약품,000020
59,2018-01-03,10000,9900,10250,9820,268220,동화약품,000020
58,2018-01-04,9750,10050,10050,9680,161342,동화약품,000020
57,2018-01-05,9910,9750,9980,9750,116604,동화약품,000020
56,2018-01-08,9950,10000,10150,9940,158326,동화약품,000020
...,...,...,...,...,...,...,...,...
1,2023-03-21,8960,9250,9280,8950,57209,코오롱티슈진,950160
0,2023-03-22,8900,8980,9050,8850,67590,코오롱티슈진,950160
0,2023-03-22,8900,8980,9050,8850,67590,코오롱티슈진,950160
0,2023-03-22,8900,8980,9050,8850,67590,코오롱티슈진,950160


In [13]:
res2.to_csv('../../../df_230323.csv')

In [65]:
sss = set(res['종목명'].to_list())

In [74]:
tt = res.groupby(by='종목명', group_keys=True).apply(lambda x: x)

In [ ]:
tt

In [78]:
tt.to_csv('kospi200_groub.csv')

In [34]:
df = pd.concat([df_1q,df_2q,df_3q,df_4q])
df.columns =['날짜','종가','시가','최고가','최저가','누적거래량','종목명']
df['날짜'] = pd.to_datetime(df['날짜'])
df.sort_values(by='날짜',inplace=True)

In [36]:
df

,날짜,종가,시가,최고가,최저가,누적거래량,종목명
58,2022-01-03,143500,143500,146000,142500,15765,BGF리테일
57,2022-01-04,142000,142500,143500,140500,29610,BGF리테일
56,2022-01-05,139500,140500,141500,138000,48402,BGF리테일
55,2022-01-06,141500,139000,143500,138500,36533,BGF리테일
54,2022-01-07,140000,143500,144500,139500,28464,BGF리테일
...,...,...,...,...,...,...,...
4,2022-12-23,206500,211000,211000,204500,28853,BGF리테일
3,2022-12-26,204500,206000,208000,201000,33859,BGF리테일
2,2022-12-27,208500,204500,209000,202000,43241,BGF리테일
1,2022-12-28,212000,205500,214000,202500,28229,BGF리테일


In [35]:
df.iloc[:,1:6] = df.iloc[:,1:6].astype('int')

C:\Users\SCH\AppData\Local\Temp\ipykernel_16256\3874453458.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,1:6] = df.iloc[:,1:6].astype('int')


In [191]:
# df.to_csv('df_example.csv')